In [11]:
#!/usr/bin/env python
"""
Tutorial to demonstrate running parameter estimation on a binary neutron star
system taking into account tidal deformabilities with a physically motivated
model for the tidal deformabilities.

WARNING: The code is extremely slow.
"""


import bilby
import numpy as np
from bilby.gw.eos import EOSFamily, TabularEOS

# Specify the output directory and the name of the simulation.
outdir = "outdir"
label = "bns_inference_18.0"
bilby.core.utils.setup_logger(outdir=outdir, label=label)

# Set up a random seed for result reproducibility.  This is optional!
np.random.seed(88170235)

# We are going to inject a binary neutron star waveform.  We first establish a
# dictionary of parameters that includes all of the different waveform
# parameters, including masses of the two black holes (mass_1, mass_2),
# aligned spins of both black holes (chi_1, chi_2), etc.

# We're also going to 'inject' the MPA1 equation of state.
# This is done by injecting masses for the two neutron-stars,
# assuming a specific equation of state, and calculating
# corresponding tidal deformability parameters from the EoS and
# masses.
mpa1_eos = TabularEOS("MPA1")
mpa1_fam = EOSFamily(mpa1_eos)

mass_1 = 1.5
mass_2 = 1.3
lambda_1 = mpa1_fam.lambda_from_mass(mass_1)
lambda_2 = mpa1_fam.lambda_from_mass(mass_2)

In [12]:
print('lambda_1 =', lambda_1, 'lambda_2 =', lambda_2, 
      'chirp_mass =', (mass_1*mass_2)**(3/5)/(mass_1+mass_2)**(1/5), 
      'symmetric_mass_ratio =', (mass_1*mass_2)/(mass_1+mass_2)**2, 
      'mass_ratio = ', mass_2/mass_1
)

lambda_1 = 342.4622864681741 lambda_2 = 779.6186030639271 chirp_mass = 1.2150360414642816 symmetric_mass_ratio = 0.24872448979591844 mass_ratio =  0.8666666666666667


In [13]:
injection_parameters = dict(
    mass_1=mass_1,
    mass_2=mass_2,
    chi_1=0.02,
    chi_2=0.02,
    luminosity_distance=200.0,
    theta_jn=0.4,
    psi=2.659,
    phase=1.3,
    geocent_time=1126259642.413,
    ra=1.375,
    dec=-1.2108,
    lambda_1=lambda_1,
    lambda_2=lambda_2,
)

# Set the duration and sampling frequency of the data segment that we're going
# to inject the signal into. For the
# TaylorF2 waveform, we cut the signal close to the isco frequency
duration = 32
sampling_frequency = 2048
start_time = injection_parameters["geocent_time"] + 2 - duration

In [14]:
# Fixed arguments passed into the source model. The analysis starts at 40 Hz.
# Note that the EoS sampling is agnostic to waveform model as long as the approximant
# can include tides.
waveform_arguments = dict(
    waveform_approximant="IMRPhenomPv2_NRTidal",
    reference_frequency=50.0,
    minimum_frequency=40.0,
)

# Create the waveform_generator using a LAL Binary Neutron Star source function
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_neutron_star,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters,
    waveform_arguments=waveform_arguments,
)

10:50 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_neutron_star
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters


In [15]:
# Set up interferometers.  In this case we'll use three interferometers
# (LIGO-Hanford (H1), LIGO-Livingston (L1), and Virgo (V1)).
# These default to their design sensitivity and start at 40 Hz.
interferometers = bilby.gw.detector.InterferometerList(["H1", "L1", "V1"])
for interferometer in interferometers:
    interferometer.minimum_frequency = 40
interferometers.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency, duration=duration, start_time=start_time
)
interferometers.inject_signal(
    parameters=injection_parameters, waveform_generator=waveform_generator
)

10:50 bilby INFO    : Injected signal in H1:
10:50 bilby INFO    :   optimal SNR = 8.61
10:50 bilby INFO    :   matched filter SNR = 9.94+0.42j
10:50 bilby INFO    :   mass_1 = 1.5
10:50 bilby INFO    :   mass_2 = 1.3
10:50 bilby INFO    :   chi_1 = 0.02
10:50 bilby INFO    :   chi_2 = 0.02
10:50 bilby INFO    :   luminosity_distance = 200.0
10:50 bilby INFO    :   theta_jn = 0.4
10:50 bilby INFO    :   psi = 2.659
10:50 bilby INFO    :   phase = 1.3
10:50 bilby INFO    :   geocent_time = 1126259642.413
10:50 bilby INFO    :   ra = 1.375
10:50 bilby INFO    :   dec = -1.2108
10:50 bilby INFO    :   lambda_1 = 342.4622864681741
10:50 bilby INFO    :   lambda_2 = 779.6186030639271
10:50 bilby INFO    : Injected signal in L1:
10:50 bilby INFO    :   optimal SNR = 6.97
10:50 bilby INFO    :   matched filter SNR = 7.47+1.42j
10:50 bilby INFO    :   mass_1 = 1.5
10:50 bilby INFO    :   mass_2 = 1.3
10:50 bilby INFO    :   chi_1 = 0.02
10:50 bilby INFO    :   chi_2 = 0.02
10:50 bilby INFO    

[{'plus': array([0.00000000e+00-0.00000000e+00j, 0.00000000e+00-0.00000000e+00j,
         0.00000000e+00-0.00000000e+00j, ...,
         1.06432054e-25+1.59358181e-26j, 1.06421968e-25+1.59732981e-26j,
         0.00000000e+00-0.00000000e+00j]),
  'cross': array([0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, ...,
         1.58820935e-26-1.06073238e-25j, 1.59194472e-26-1.06063187e-25j,
         0.00000000e+00+0.00000000e+00j])},
 {'plus': array([0.00000000e+00-0.00000000e+00j, 0.00000000e+00-0.00000000e+00j,
         0.00000000e+00-0.00000000e+00j, ...,
         1.06432054e-25+1.59358181e-26j, 1.06421968e-25+1.59732981e-26j,
         0.00000000e+00-0.00000000e+00j]),
  'cross': array([0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, ...,
         1.58820935e-26-1.06073238e-25j, 1.59194472e-26-1.06063187e-25j,
         0.00000000e+00+0.00000000e+00j])},
 {'plus': array([0.00000

In [26]:
# We're going to sample in chirp_mass, symmetric_mass_ratio, and
# specific EoS model parameters. We're using a 4-parameter
# spectrally-decomposed EoS parameterization from Lindblom (2010).
# BNS have aligned spins by default, if you want to allow precessing spins
# pass aligned_spin=False to the BNSPriorDict
priors = bilby.gw.prior.BNSPriorDict()
for key in [
    "psi",
    "geocent_time",
    "ra",
    "dec",
    "chi_1",
    "chi_2",
    "theta_jn",
    "luminosity_distance",
    "phase",
]:
    priors[key] = injection_parameters[key]
for key in ["mass_1", "mass_2", "mass_ratio"]:
    del priors[key]

#priors["mass_ratio"] = mass_2/mass_1
#priors['mass_1'] = bilby.core.prior.Constraint(
#    1, 2, name="mass_1"
#)
#priors['mass_2'] = bilby.core.prior.Constraint(
#    1, 2, name="mass_2"
#)
priors["chirp_mass"] = bilby.core.prior.Uniform(
    0.4, 3.4, name="chirp_mass", unit="$M_{\\odot}$"
)
#priors["symmetric_mass_ratio"] = bilby.core.prior.Uniform(
#    0.1, 0.4, name="symmetric_mass_ratio"
#)
priors["lambda_1"] = bilby.core.prior.Uniform(
    0, 2000, name="lambda_1"
)
priors["lambda_2"] = bilby.core.prior.Uniform(
    0, 2000, name="lambda_2"
)
priors["mass_ratio"] = bilby.core.prior.Uniform(
    0.5, 1, name="mass_ratio"
)

10:57 bilby INFO    : No prior given, using default BNS priors in /home/iuliu/anaconda3/envs/work/lib/python3.9/site-packages/bilby/gw/prior_files/aligned_spins_bns_tides_on.prior.


In [27]:
priors

{'chirp_mass': Uniform(minimum=0.4, maximum=3.4, name='chirp_mass', latex_label='$\\mathcal{M}$', unit='$M_{\\odot}$', boundary=None),
 'luminosity_distance': 200.0,
 'dec': -1.2108,
 'ra': 1.375,
 'theta_jn': 0.4,
 'psi': 2.659,
 'phase': 1.3,
 'chi_1': 0.02,
 'chi_2': 0.02,
 'lambda_1': Uniform(minimum=0, maximum=2000, name='lambda_1', latex_label='$\\Lambda_1$', unit=None, boundary=None),
 'lambda_2': Uniform(minimum=0, maximum=2000, name='lambda_2', latex_label='$\\Lambda_2$', unit=None, boundary=None),
 'geocent_time': 1126259642.413,
 'mass_ratio': Uniform(minimum=0.5, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None)}

In [28]:
# The eos_check prior imposes several hard physical constraints on samples like
# enforcing causality and monotinicity of the EoSs. In almost ever conceivable
# sampling scenario, this should be enabled.
priors["eos_check"] = bilby.gw.prior.EOSCheck()

# Initialise the likelihood by passing in the interferometer data (IFOs)
# and the waveform generator
likelihood = bilby.gw.GravitationalWaveTransient(
    interferometers=interferometers,
    waveform_generator=waveform_generator,
)

In [ ]:
#Run sampler.  In this case we're going to use the `dynesty` sampler
result = bilby.run_sampler(
    likelihood=likelihood,
    priors=priors,
    sampler="dynesty",
    npoints=150,
    walks=50,
    npool=8,
    injection_parameters=injection_parameters,
    outdir=outdir,
    label=label,
    conversion_function=bilby.gw.conversion.generate_all_bns_parameters,
    resume=True,
)

result.plot_corner()

10:57 bilby INFO    : Running for label 'bns_inference_18.0', output will be saved to 'outdir'
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
10:57 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redund

0it [00:00, ?it/s]

10:57 bilby INFO    : Using sampler Dynesty with kwargs {'bound': 'multi', 'sample': 'rwalk', 'verbose': True, 'periodic': None, 'reflective': None, 'check_point_delta_t': 1800, 'nlive': 150, 'first_update': None, 'walks': 50, 'npdim': None, 'rstate': None, 'queue_size': 8, 'pool': None, 'use_pool': None, 'live_points': None, 'logl_args': None, 'logl_kwargs': None, 'ptform_args': None, 'ptform_kwargs': None, 'enlarge': 1.5, 'bootstrap': None, 'vol_dec': 0.5, 'vol_check': 8.0, 'facc': 0.2, 'slices': 5, 'update_interval': 90, 'print_func': <bound method Dynesty._print_func of <bilby.core.sampler.dynesty.Dynesty object at 0x7efcb2bbcdf0>>, 'dlogz': 0.1, 'maxiter': None, 'maxcall': None, 'logl_max': inf, 'add_live': True, 'print_progress': True, 'save_bounds': False, 'n_effective': None, 'maxmcmc': 5000, 'nact': 5, 'print_method': 'tqdm'}
10:57 bilby INFO    : Checkpoint every check_point_delta_t = 600s
10:57 bilby INFO    : Using dynesty version 1.0.1
10:57 bilby INFO    : Using the bilby

In [10]:
print('lambda_1 =', lambda_1, 'lambda_2 =', lambda_2, 
      'chirp_mass =', (mass_1*mass_2)**(3/5)/(mass_1+mass_2)**(1/5), 
      'symmetric_mass_ratio =', (mass_1*mass_2)/(mass_1+mass_2)**2, 
      'mass_ratio = ', mass_2/mass_1
)

lambda_1 = 342.4622864681741 lambda_2 = 779.6186030639271 chirp_mass = 1.2150360414642816 symmetric_mass_ratio = 0.24872448979591844 mass_ratio =  0.8666666666666667
